In [125]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd
from functools import reduce

In [126]:
directory = '../../../Chapters/tables/'

def getBaseProjectData():
    """
    Gets the base data for the project

    Returns:
    A dataframe with user activities
    """
    systemsData = pd.read_csv("../data/uat_raw/systems.csv")
    projectBaseData = pd.read_csv("../data/uat_raw/uat1.csv")
    users = pd.read_csv("../data/uat_raw/users.csv")

    temp = pd.merge(systemsData, projectBaseData,on='SubSystemId')
    temp = pd.merge(users, temp, on='UserId')
    return temp.loc[temp['Project'] == 1], temp.loc[temp['Project'] == 2];

def getNormilisedBySecondCriteria(data, secondFilter):
    temp = data.groupby(['SubSystemId',secondFilter]).size().reset_index(name='Count')
    pivoted_df = temp.pivot_table(index='SubSystemId', columns=secondFilter,
     values='Count', fill_value=0)
    return pivoted_df

def getNormalisedByTotal(data):
    temp = data.groupby(['SubSystemId']).agg(Priority=('Priority', 'mean'), Count=('SubSystemId', 'count')).reset_index()
    temp['NormalisedCount'] = temp['Count'].apply(lambda x: 
        (x-temp['Count'].min())/(temp['Count'].max()-temp['Count'].min()))

    temp['MaintenanceFactor'] = temp.apply(lambda row: row['NormalisedCount'] * row['Priority'], axis=1)
    return temp

In [127]:
# Get project actvities
projectA, projectB = getBaseProjectData()

# Get project total activities per project
projectA_Normilised = getNormalisedByTotal(projectA)
projectB_Normilised = getNormalisedByTotal(projectA)
projectA_ByActivityType = getNormilisedBySecondCriteria(projectA, 'ActivityType')
projectB_ByActivityType = getNormilisedBySecondCriteria(projectB, 'ActivityType')

In [128]:
def createLatexRows(data, filename, caption):
    latex_rows = ''
    for index, row in data.iterrows():
        latex_rows += '\n {:d} & {:d} & {:d} & {:.4f} & {:.4f} \\\ \\hline'.format(int(row['SubSystemId']), int(row['Priority']), int(row['Count']), row['NormalisedCount'], row['MaintenanceFactor'])

    header = r"""
    \begin{xltabular}{\textwidth}{|X|X|X|X|X|}
        \caption[%s]
        {\textit{%s}}
        \label{tbl:apx_%s} \\
        
        \hline
        \textbf{Subsystem} & \textbf{Priorty}  & \textbf{Actvities} & \textbf{Normalised Actvity} & \textbf{Maintenance Actvity} \\
        \hline
        \endfirsthead

        \multicolumn{5}{c}
        {\tablename\ \thetable{} -- continued from previous page} \\
        \hline
        \textbf{Subsystem} & \textbf{Priorty}  & \textbf{Actvities} & \textbf{Normalised Actvity} & \textbf{Maintenance Actvity} \\ 
        \endhead

        \multicolumn{5}{|r|}{{Continued on next page}} \\ \hline
        \endfoot

        \hline
        \endlastfoot
    """ % (caption, caption, filename) 
    
    table = header + latex_rows.strip() + r"""
    \end{xltabular}
    """

    filepath = directory + filename + '.tex'

    # Write the latex_rows string to the file
    with open(filepath, 'w') as f:
        f.write(table)


createLatexRows(projectA_Normilised, "projectA_Normilised", "Case study A results")
